In [4]:
import time
import dlib
import pickle

import numpy as np
import pandas as pd

from cv2 import cv2
from tqdm import tqdm, trange
from threading import Thread
from keras.models import load_model
from sklearn.preprocessing import Normalizer

Using TensorFlow backend.


In [5]:
dlib_detector = dlib.get_frontal_face_detector()
x_encoder = Normalizer(norm='l2')
result = []

In [6]:
model = load_model('model/facenet_keras.h5')
model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_Ba

c:\programdata\anaconda3\envs\ta\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [7]:
classifier_file = open('model/classifier.pkl', 'rb')
classifier = pickle.load(classifier_file)
classifier_file.close()
print(classifier)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


In [8]:
label_encoding_file = open('label_encoding.pkl', 'rb')
label_encoding = pickle.load(label_encoding_file)
label_encoding_file.close()
print(label_encoding)

{0: 'afif', 1: 'aldi', 2: 'bapaks2', 3: 'bu nanik', 4: 'dandy', 5: 'dinial', 6: 'fadli', 7: 'fandykun', 8: 'felix', 9: 'ferdinand', 10: 'ibucs', 11: 'kc', 12: 'mala', 13: 'randi', 14: 'rangga', 15: 'raras', 16: 'safhira', 17: 'sherly', 18: 'syauqi', 19: 'timo', 20: 'titut', 21: 'wildan', 22: 'yoshima', 23: 'yulia', 24: 'zakiya'}


In [33]:
def worker(frame):
    global result, model
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = dlib_detector(gray, 0)
    
    result = []
    
    for item in rects:
        try:
            x1 = item.left()
            y1 = item.top()
            x2 = item.right()
            y2 = item.bottom()

            segmented_face = frame[y1:y2, x1:x2].copy()
            segmented_face = cv2.resize(segmented_face, (160, 160))
            x = segmented_face.astype('float32')
            mean, std = x.mean(), x.std()
            x = (x-mean) / std
            x = np.expand_dims(x, axis=0)
            feature = model.predict(x)
            encoded_face = x_encoder.transform(np.array([feature[0]]))
            predicted = classifier.predict(encoded_face)
            pred_class = label_encoding[predicted[0]]
            result.append(((x1, y1, x2, y2), pred_class))
        except Exception as e:
            print(str(e))

In [82]:
video = 'data/data.mp4'
capturer = cv2.VideoCapture(video)
capturer.set(1, 19500)
output = cv2.VideoWriter('data/result/data_result.mp4', -1, 15.0, (1077, 720))
frame_time = []
counter = 0
while True:
    try:
        now = time.time()
        ret, frame = capturer.read()
        if not ret:
            break
#         frame = cv2.resize(frame, (1920, 1080))
        frame = cv2.resize(frame, (1280, 720))
#         frame = cv2.resize(frame, (854, 480))
#         frame = cv2.resize(frame, (640, 360))

        frame = frame[:,0:1077]
#         frame = frame[:,0:819]

        if counter % 5 == 0:
            worker(frame.copy())
        
        
        counter += 1
        time.sleep(1/30)
        for item in result:
            cv2.rectangle(frame, (item[0][0], item[0][1]), (item[0][2], item[0][3]), (255, 255, 255), 2)
            cv2.putText(frame, item[1], (item[0][0], item[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        if len(frame_time) != 0:
            total_time = sum(frame_time)
#             cv2.putText(frame, str(counter), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, str(int(len(frame_time)/total_time)) + ' fps', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        output.write(frame)
        cv2.imshow('video', frame)
        frame_time.append(time.time()-now)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(str(e))
        break

cv2.destroyAllWindows()
capturer.release()
output.release()